In [3]:
import os
import skimage 
import numpy as np
from tqdm import tqdm
from tqdm.contrib import itertools as it

from pylibCZIrw import czi as pyczi
from cztile.fixed_total_area_strategy import AlmostEqualBorderFixedTotalAreaStrategy2D


import matplotlib.pyplot as plt
# from scroutines.config_plots import *

/home/qlyu/mambaforge/envs/czi2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

ddir = '/scr/qlyu/projects/v1/data/multifish_demo_tiny/inputs'
f_r3v1 = os.path.join(ddir, 'LHA3_R3_tiny_V01.czi') 
f_r3v2 = os.path.join(ddir, 'LHA3_R3_tiny_V02.czi') 
f_r5v1 = os.path.join(ddir, 'LHA3_R5_tiny_V01.czi') 
f_r5v2 = os.path.join(ddir, 'LHA3_R5_tiny_V02.czi') 

f_r3m  = os.path.join(ddir, 'LHA3_R3_tiny.mvl') 
f_r5m  = os.path.join(ddir, 'LHA3_R5_tiny.mvl') 


In [5]:
f_all = [
    os.path.join(ddir, 'LHA3_R3_tiny_V01.czi'),
    os.path.join(ddir, 'LHA3_R3_tiny_V02.czi'), 
    os.path.join(ddir, 'LHA3_R5_tiny_V01.czi'),
    os.path.join(ddir, 'LHA3_R5_tiny_V02.czi'),
]

In [6]:
def tile_one_file(f_src, f_dst_all=[]):
    # create a "tile" 
    tiler = AlmostEqualBorderFixedTotalAreaStrategy2D(
        total_tile_width=350,
        total_tile_height=500,
        min_border_width=10,
    )

    # create CZI instance to read some metadata 
    with pyczi.open_czi(f_src) as czidoc_r:

        # get the size of the bounding rectange for the scence
        print(czidoc_r.total_bounding_box)
        print(czidoc_r.total_bounding_rectangle)
        print(czidoc_r.scenes_bounding_rectangle)
        bbox = czidoc_r.total_bounding_box
        numZ, numC = bbox['Z'][1], bbox['C'][1]
        tiles = tiler.tile_rectangle(czidoc_r.scenes_bounding_rectangle[0])

    # show the created tile locations
    for i, tile in enumerate(tiles):
        print(tile.roi)
        outdir = os.path.join(ddir, f't{i}')
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        f_dst = os.path.join(outdir, os.path.basename(f_src)) #.replace('.czi', f'_t{i}.czi'))
        f_dst_all.append(f_dst)

    with pyczi.open_czi(f_src) as czidoc_r:
        # loop over all tiles 
        for i, tile in enumerate(tiles):
            f_dst = f_dst_all[i]
            print(f"tile {i}, {f_dst}")
            with pyczi.create_czi(f_dst, exist_ok=True) as czidoc_w:
                for z, c in it.product(range(numZ), range(numC)):
                    plane = {'Z': z, 'C': c}

                    # read and write plane by plane for the same tile
                    tile2d = czidoc_r.read(
                        plane=plane,
                        roi=(
                            tile.roi.x,
                            tile.roi.y,
                            tile.roi.w,
                            tile.roi.h
                        )
                    )
                    czidoc_w.write(
                        tile2d,
                        plane=plane,
                        location=(
                            tile.roi.x,
                            tile.roi.y
                        )
                    )

In [7]:
# for f_src in f_all:
#     print(f".....processing: {f_src}.............")        
#     tile_one_file(f_src, f_dst_all=[])

In [17]:
!cat $f_r3m

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<MultiViewList>
  <Comment>
    <!--Positionlist for ZEN Multiview.-->
  </Comment>
  <Data ListName="MultiViewList">
    <Entry0 PositionX="398.811" PositionY="16317.36" PositionZ="-2584.986" PositionR="9.988056" StackRange="63.235" StackSlices="151" StackStepSize="0.421567" AcquisitionCameraFormat="0" AcquisitionZoom="1" AcquisitionFrameX="0" AcquisitionFrameY="0" AcquisitionFrameWidth="660" AcquisitionFrameHeight="920" Group="1">
      <Track1 LightSheetOffsetLeft="-0.46296" LightSheetOffsetRight="0.46846" LightSheetOffsetLeftUI="-1.99" LightSheetOffsetRightUI="-0.95">
        <LaserLines>
          <Line1 Line="488" Power="8" />
        </LaserLines>
      </Track1>
      <Track2 LightSheetOffsetLeft="-0.46404" LightSheetOffsetRight="0.46724" LightSheetOffsetLeftUI="-1.57" LightSheetOffsetRightUI="-4.33">
        <LaserLines>
          <Line1 Line="405" Power="4" />
        </LaserLines>
      </Track2>
    </Entry0>
    <Entr

In [8]:
f_new = os.path.join(ddir, 't1', 'LHA3_R3_tiny_V01.czi') 
f_new

'/scr/qlyu/projects/v1/data/multifish_demo_tiny/inputs/t1/LHA3_R3_tiny_V01.czi'

In [14]:

with pyczi.open_czi(f_new) as czidoc_r:
    # get the size of the bounding rectange for the scence
    print(czidoc_r.total_bounding_box)
    print(czidoc_r.total_bounding_rectangle)
    print(czidoc_r.scenes_bounding_rectangle)
    meta = czidoc_r.metadata
    bbox = czidoc_r.total_bounding_box
    
meta

{'T': (0, 1), 'Z': (0, 151), 'C': (0, 2), 'X': (0, 350), 'Y': (420, 920)}
Rectangle(x=0, y=420, w=350, h=500)
{0: Rectangle(x=0, y=420, w=350, h=500)}


{'ImageDocument': {'Metadata': {'Information': {'Image': {'SizeX': '350',
     'SizeY': '500',
     'SizeZ': '151',
     'SizeC': '2',
     'SizeT': '1',
     'SizeS': '1',
     'SizeM': '1',
     'Dimensions': {'Channels': {'Channel': [{'@Id': 'Channel:0',
         '@Name': '',
         'PixelType': 'Gray16'},
        {'@Id': 'Channel:1', '@Name': '', 'PixelType': 'Gray16'}]}},
     'PixelType': 'Gray16'},
    'Document': {'Title': 'CZI', 'Comment': 'pylibCZIrw generated'},
    'Application': {'Name': 'pylibCZIrw', 'Version': '3.3.1'}},
   'Scaling': {'Items': {'Distance': [{'@Id': 'X', 'Value': '0'},
      {'@Id': 'Y', 'Value': '0'},
      {'@Id': 'Z', 'Value': '0'}]}},
   'DisplaySetting': {'Channels': {'Channel': None}}}}}

In [15]:
with pyczi.open_czi(f_r3v1) as czidoc_r:
    # get the size of the bounding rectange for the scence
    print(czidoc_r.total_bounding_box)
    print(czidoc_r.total_bounding_rectangle)
    print(czidoc_r.scenes_bounding_rectangle)
    meta = czidoc_r.metadata
    bbox = czidoc_r.total_bounding_box
    
meta

{'T': (0, 1), 'Z': (0, 151), 'C': (0, 2), 'R': (0, 1), 'I': (0, 1), 'H': (0, 1), 'V': (0, 1), 'X': (0, 660), 'Y': (0, 920)}
Rectangle(x=0, y=0, w=660, h=920)
{0: Rectangle(x=0, y=0, w=660, h=920)}


{'ImageDocument': {'Metadata': {'Version': '1.0',
   'Experiment': {'ExperimentBlockIndex': '0',
    'ExperimentBlocks': {'AcquisitionBlock': {'AcquisitionModeSetup': {'BiDirectional': 'true',
       'BiDirectionalZ': 'false',
       'BitsPerSample': '16',
       'CameraBinning': '1',
       'CameraFrameHeight': '1920',
       'CameraFrameOffsetX': '0',
       'CameraFrameOffsetY': '0',
       'CameraFrameWidth': '1920',
       'CameraSuperSampling': '0',
       'DimensionT': '10',
       'DimensionX': '512',
       'DimensionY': '512',
       'DimensionZ': '1574',
       'FilterMethod': 'Average',
       'FilterMode': 'Frame',
       'FilterSamplingNumber': '1',
       'FitFramesizeToRoi': 'false',
       'HdrEnabled': 'false',
       'HdrImagingMode': '0',
       'HdrIntensity': '1',
       'HdrNumFrames': '1',
       'InterpolationY': '1',
       'Objective': 'W Plan-Apochromat 20x/1.0 UV-VIS_4909000187',
       'OffsetX': '0',
       'OffsetY': '0',
       'OffsetZ': '0.00017662100